In [1]:
import climate_learn as cl
from climate_learn.data import DataModule
from climate_learn.data.climate_dataset.args import ERA5Args
from climate_learn.data.dataset.args import ShardDatasetArgs
from climate_learn.data.task.args import ForecastingArgs

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = "/home/data/datasets/weatherbench/era5/5.625deg"
dataset = "era5"
variables = [
    "2m_temperature",
    # "geopotential",
    # "temperature",
    # "specific_humidity",
    # "u_component_of_wind",
    # "v_component_of_wind"
]
in_vars = [f"{dataset}:{var}" for var in variables]
out_vars = [f"{dataset}:{var}" for var in variables]

total_years = range(1979, 2018)
train_years = range(1979, 2015)
val_years = range(2015, 2017)
test_years = range(2017, 2019)
history = 3
subsample = 6
pred_range = 6  # CHANGE ME

In [3]:
# General arguments
data_args = ERA5Args(root, variables, total_years, name=dataset)
forecasting_args = ForecastingArgs(
    in_vars,
    out_vars,
    history=history,
    pred_range=pred_range,
    subsample=subsample
)
shard_dataset_args = ShardDatasetArgs(data_args, forecasting_args, 5)

# Split-specific arguments
train_dataset_args = shard_dataset_args.create_copy(
    {"climate_dataset_args": {"years": train_years}}
)
val_dataset_args = shard_dataset_args.create_copy(
    {"climate_dataset_args": {"years": val_years}, "n_chunks": 1}
)
test_dataset_args = shard_dataset_args.create_copy(
    {"climate_dataset_args": {"years": test_years}, "n_chunks": 1}
)

# Data module
dm = DataModule(
    train_dataset_args,
    val_dataset_args,
    test_dataset_args,
    batch_size=32,
    num_workers=8
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 37.63it/s]


In [4]:
climatology = cl.load_forecasting_module(data_module=dm, preset="climatology")
persistence = cl.load_forecasting_module(data_module=dm, preset="persistence")
linreg = cl.load_forecasting_module(data_module=dm, preset="linear-regression")

Loading preset: climatology


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 20.44it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading preset: persistence


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.58it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading preset: linear-regression


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.08it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize


In [5]:
trainer = cl.Trainer(early_stopping="lat_rmse:aggregate", patience=5)

Global seed set to 0
/home/jason.jewik/climate-learn/src/climate_learn/trainer.py:48: UserWarning: In interactive environment: cannot use DDP spawn strategy
  warn("In interactive environment: cannot use DDP spawn strategy")
/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/trainer/setup.py:179: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=10)`.
  category=PossibleUserWarning,


In [ ]:
trainer.fit(linreg, data_module)

In [6]:
trainer.test(climatology, dm)

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Fatal Python error: Cannot recover from stack overflow.

Current thread 0x00007f1946073740 (most recent call first):
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/control.py", line 198 in strip_control_codes
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/text.py", line 139 in __init__
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/text.py", line 1081 in <genexpr>
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/containers.py", line 70 in __init__
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/text.py", line 1081 in divide
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/text.py", line 1184 in wrap
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/rich/text.py", line 660 in __rich_console__
  File "/home/jason.jewik/miniconda3/envs/climate/lib/python

RuntimeError: DataLoader worker (pid(s) 182210) exited unexpectedly

In [7]:
trainer.test(persistence, dm)

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/trainer/call.py:48: 
UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

In [ ]:
trainer.test(linreg, dm)